# Data Cleaning Task

In [1]:
# Step 1 - load
from pathlib import Path
import re
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

DATA_DIR = Path("data")
assert DATA_DIR.exists()
print("Using data dir:", DATA_DIR.resolve())

customers = pd.read_csv(DATA_DIR / "customers.csv")
products  = pd.read_csv(DATA_DIR / "products.csv")
orders    = pd.read_csv(DATA_DIR / "orders.csv")

print("Shapes →",
      "customers:", customers.shape,
      "| products:", products.shape,
      "| orders:", orders.shape)

display(customers.head())
display(products.head())
display(orders.head())

print("\nInfo — customers:"); customers.info()
print("\nInfo — products:");  products.info()
print("\nInfo — orders:");    orders.info()

Using data dir: /Users/bilalarshad/myown/ML/ai-engineer-test/data
Shapes → customers: (100, 5) | products: (50, 5) | orders: (200, 7)


,Customer ID,fullName,e-mail,SignUpDate,vip
0,C001,Norma Fisher,tammy76@example.com,"August 14, 2025",NaN
1,C002,Steven Robinson,juancampos@example.net,13/06/2025,NaN
2,C003,MS. MICHELE GUZMAN,kyleblair@example.net,23/12/2024,Y
3,C004,Heather Stewart,tammywoods@example.net,"November 28, 2024",N
4,C005,KIMBERLY SMITH,ypage@example.net,"April 10, 2025",Yes


,sku,name,category,unit_price_usd,weight_kg
0,SKU-001,Front,Accessories,"49,8",0.26
1,SKU-002,Learn,Widgets,$70.26,0.59
2,SKU-003,Both,Tools,$5.61,1.65
3,SKU-004,Onto,Widgets,99.97,0.99
4,SKU-005,We,Gadgets,"18,36",0.11


,order_id,custId,product_sku,qty,unit_price_usd,order_date,ship_country
0,1000,C032,SKU-047,NaN,$27.36,2025-06-08T01:02:36Z,GT
1,1001,c070,sku-023,five,$5.61,"June 28, 2025 03:50",MA
2,1002,C008,SKU-009,4,"74,33","June 04, 2025 18:03",JM
3,1003,C096,SKU-017,4,"27,91",06/04/2025 10:03,GY
4,1004,C085,sku-010,five,70.26,08/03/2025 19:19,KP



Info — customers:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer ID  100 non-null    object
 1   fullName     100 non-null    object
 2   e-mail       100 non-null    object
 3   SignUpDate   96 non-null     object
 4   vip          87 non-null     object
dtypes: object(5)
memory usage: 4.0+ KB

Info — products:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sku             50 non-null     object 
 1   name            50 non-null     object 
 2   category        50 non-null     object 
 3   unit_price_usd  46 non-null     object 
 4   weight_kg       50 non-null     float64
dtypes: float64(1), object(4)
memory usage: 2.1+ KB

Info — orders:
<class 'pandas.core.frame.DataFrame'>
RangeIndex

In [2]:
# Step 2 - Standardize columns

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(r"[^a-z0-9]+", "_", regex=True)
        .str.strip("_")
    )
    return df

def strip_strings(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    obj_cols = df.select_dtypes(include=["object"]).columns
    for c in obj_cols:
        df[c] = df[c].astype(str).str.strip()
    return df

customers = normalize_columns(customers)
products  = normalize_columns(products)
orders    = normalize_columns(orders)

customers = customers.rename(columns={
    "customer id": "customer_id",
    "custid": "customer_id",
    "e_mail": "email",
    "e-mail": "email",
    "signupdate": "signup_date",
    "fullname": "full_name",
})
orders = orders.rename(columns={
    "custid": "customer_id",
})

customers = strip_strings(customers)
products  = strip_strings(products)
orders    = strip_strings(orders)

if "sku" in products.columns and "product_sku" not in products.columns:
    products = products.rename(columns={"sku": "product_sku"})
if "product_sku" in products.columns:
    products["product_sku"] = products["product_sku"].str.upper()
if "product_sku" in orders.columns:
    orders["product_sku"] = orders["product_sku"].str.upper()

customers.head()


,customer_id,full_name,email,signup_date,vip
0,C001,Norma Fisher,tammy76@example.com,"August 14, 2025",nan
1,C002,Steven Robinson,juancampos@example.net,13/06/2025,nan
2,C003,MS. MICHELE GUZMAN,kyleblair@example.net,23/12/2024,Y
3,C004,Heather Stewart,tammywoods@example.net,"November 28, 2024",N
4,C005,KIMBERLY SMITH,ypage@example.net,"April 10, 2025",Yes


In [3]:
# Step 3 — Clean customers

def clean_customers(df: pd.DataFrame) -> pd.DataFrame:
    c = df.copy()

    for col in c.select_dtypes(include=["object"]).columns:
        c[col] = c[col].astype(str).str.strip()

    if "full_name" in c.columns:
        c["full_name"] = (
            c["full_name"]
            .str.replace(r"\s+", " ", regex=True)
            .str.title()
        )

    if "vip" in c.columns:
        vip_map = {"yes": True, "y": True, "no": False, "n": False, "": pd.NA, "nan": pd.NA}
        c["vip"] = (
            c["vip"].astype(str).str.strip().str.lower().map(vip_map)
        )

    if "signup_date" in c.columns:
        c["signup_date"] = pd.to_datetime(
            c["signup_date"],
            errors="coerce",
            dayfirst=True,
        )

    return c

customers = clean_customers(customers)
customers.head()


,customer_id,full_name,email,signup_date,vip
0,C001,Norma Fisher,tammy76@example.com,2025-08-14,<NA>
1,C002,Steven Robinson,juancampos@example.net,NaT,<NA>
2,C003,Ms. Michele Guzman,kyleblair@example.net,NaT,True
3,C004,Heather Stewart,tammywoods@example.net,2024-11-28,False
4,C005,Kimberly Smith,ypage@example.net,2025-04-10,True


In [4]:
# Step 4 — Clean products

def to_num_with_comma(s):
    s = str(s).strip()
    if s == "" or s.lower() == "nan":
        return pd.NA
    s = re.sub(r"[^\d,.\-]", "", s)

    if "," in s and "." not in s:
        s = s.replace(".", "").replace(",", ".")
    else:
        s = s.replace(",", "")

    return pd.to_numeric(s, errors="coerce")

def clean_products(df: pd.DataFrame) -> pd.DataFrame:
    p = df.copy()
    if "unit_price_usd" in p.columns:
        p["unit_price_usd"] = p["unit_price_usd"].map(to_num_with_comma).astype("Float64")
    return p

products = clean_products(products)
products.head()


,product_sku,name,category,unit_price_usd,weight_kg
0,SKU-001,Front,Accessories,49.8,0.26
1,SKU-002,Learn,Widgets,70.26,0.59
2,SKU-003,Both,Tools,5.61,1.65
3,SKU-004,Onto,Widgets,99.97,0.99
4,SKU-005,We,Gadgets,18.36,0.11


In [5]:
# Step 5 — Clean orders

num_words = {
    "one":1, "two":2, "three":3, "four":4, "five":5,
    "six":6, "seven":7, "eight":8, "nine":9, "ten":10
}

def clean_orders(df: pd.DataFrame) -> pd.DataFrame:
    o = df.copy()

    if "customer_id" in o.columns:
        o["customer_id"] = o["customer_id"].astype(str).str.strip().str.upper()
    if "product_sku" in o.columns:
        o["product_sku"] = o["product_sku"].astype(str).str.strip().str.upper()

    if "order_date" in o.columns:
        parsed_dates = []
        for idx, date_str in o["order_date"].items():
            if pd.isna(date_str):
                parsed_dates.append(pd.NaT)
                continue
            
            date_str = str(date_str).strip()
            parsed = pd.NaT
            
            if "T" in date_str and "Z" in date_str:
                # ISO format like "2025-06-08T01:02:36Z"
                parsed = pd.to_datetime(date_str, errors="coerce", utc=True)
            elif "/" in date_str and len(date_str.split("/")[0]) == 4:
                # Format like "2025/05/13 02:57+01:00"
                parsed = pd.to_datetime(date_str, errors="coerce", utc=True)
            elif "/" in date_str:
                # DD/MM/YYYY format (European) - like "06/04/2025 10:03"
                parsed = pd.to_datetime(date_str, errors="coerce", dayfirst=True, utc=True)
            else:
                # Month name formats or other - like "June 28, 2025 03:50"
                parsed = pd.to_datetime(date_str, errors="coerce", utc=True)
            
            parsed_dates.append(parsed)
        
        o["order_date"] = pd.Series(parsed_dates, index=o.index)

    if "qty" in o.columns:
        # try numeric first
        q = pd.to_numeric(o["qty"], errors="coerce")

        #Replace words  for example 8 instead of 'eight'
        needs = q.isna() & o["qty"].notna()
        if needs.any():
            w = (
                o.loc[needs, "qty"].astype(str).str.lower().str.strip()
                  .map(num_words)
            )
            q = q.fillna(w)

        o["qty"] = pd.to_numeric(q, errors="coerce").astype("Int64")  # integer nullable

    # same logic as products unit price usd --> numeric
    if "unit_price_usd" in o.columns:
        o["unit_price_usd"] = o["unit_price_usd"].map(to_num_with_comma).astype("Float64")

    return o

orders = clean_orders(orders)

print("orders dtypes:")
display(orders.dtypes)
display(orders.head())

orders dtypes:


order_id                        int64
customer_id                    object
product_sku                    object
qty                             Int64
unit_price_usd                Float64
order_date        datetime64[ns, UTC]
ship_country                   object
dtype: object

,order_id,customer_id,product_sku,qty,unit_price_usd,order_date,ship_country
0,1000,C032,SKU-047,<NA>,27.36,2025-06-08 01:02:36+00:00,GT
1,1001,C070,SKU-023,5,5.61,2025-06-28 03:50:00+00:00,MA
2,1002,C008,SKU-009,4,74.33,2025-06-04 18:03:00+00:00,JM
3,1003,C096,SKU-017,4,27.91,2025-04-06 10:03:00+00:00,GY
4,1004,C085,SKU-010,5,70.26,2025-03-08 19:19:00+00:00,KP


In [6]:
# Step 6 — Deduplicate

from typing import Tuple

def deduplicate_data(customers: pd.DataFrame,
                     products: pd.DataFrame,
                     orders: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    c = customers.copy() if customers is not None else None
    p = products.copy()  if products  is not None else None
    o = orders.copy()    if orders    is not None else None

    # --- Customers: earliest signup_date; fill from later duplicates ---
    if c is not None and not c.empty and "customer_id" in c.columns:
        sort_cols = ["customer_id"]
        if "signup_date" in c.columns:
            sort_cols.append("signup_date")
        c = c.sort_values(sort_cols, na_position="last")

        # build an agg dict: for each column (except key), pick the first non-null
        def first_non_null(s: pd.Series):
            i = s.first_valid_index()
            return s.loc[i] if i is not None else pd.NA

        agg_map = {col: first_non_null for col in c.columns if col != "customer_id"}
        c = (c.groupby("customer_id", as_index=False).agg(agg_map))

    # --- Orders: drop exact + business duplicates ---
    if o is not None and not o.empty:
        o = o.drop_duplicates()
        keys = ["customer_id", "product_sku", "order_date", "qty", "unit_price_usd"]
        if all(k in o.columns for k in keys):
            o = o.drop_duplicates(subset=keys, keep="first")

    return c, p, o

# apply
customers, products, orders = deduplicate_data(customers, products, orders)
print("After dedup → customers:", customers.shape, "| orders:", orders.shape)


After dedup → customers: (100, 5) | orders: (200, 7)


In [7]:
# Step 7 — Integrity checks

def run_integrity_checks(customers: pd.DataFrame,
                         products: pd.DataFrame,
                         orders: pd.DataFrame):
    # >>>> Unknown SKUs: orders -> products
    if {"product_sku"}.issubset(orders.columns) and {"product_sku"}.issubset(products.columns):
        ord_sku = orders["product_sku"].astype(str).str.upper()
        prod_sku = products["product_sku"].astype(str).str.upper()
        unknown_sku_mask = ~ord_sku.isin(prod_sku)
        unknown_sku_orders = orders.loc[unknown_sku_mask]
        print(f"Unknown SKUs: {len(unknown_sku_orders)}")
        if not unknown_sku_orders.empty:
            ids = unknown_sku_orders["order_id"].tolist() if "order_id" in unknown_sku_orders.columns else unknown_sku_orders.index.tolist()
            print("Order IDs (first 20):", ids[:20])
    else:
        print("SKU check skipped (missing 'product_sku').")

    # >>>> Unknown customers: orders -> customers
    if {"customer_id"}.issubset(orders.columns) and {"customer_id"}.issubset(customers.columns):
        ord_cid = orders["customer_id"].astype(str).str.strip().str.upper()
        cust_cid = customers["customer_id"].astype(str).str.strip().str.upper()
        unknown_cust_mask = ~ord_cid.isin(cust_cid)
        unknown_cust_orders = orders.loc[unknown_cust_mask]
        print(f"Unknown customers: {len(unknown_cust_orders)}")
        if not unknown_cust_orders.empty:
            ids = unknown_cust_orders["order_id"].tolist() if "order_id" in unknown_cust_orders.columns else unknown_cust_orders.index.tolist()
            print("Order IDs (first 20):", ids[:20])
    else:
        print("Customer check skipped (missing 'customer_id').")

    # >>>> Non-negative assertions
    if "qty" in orders.columns:
        neg_qty = orders["qty"].notna() & (orders["qty"] < 0)
        assert not neg_qty.any(), f"Found negative qty in {neg_qty.sum()} rows."
    if "unit_price_usd" in orders.columns:
        neg_price = orders["unit_price_usd"].notna() & (orders["unit_price_usd"] < 0)
        assert not neg_price.any(), f"Found negative unit_price_usd in {neg_price.sum()} rows."

    print("Integrity checks completed ..")

run_integrity_checks(customers, products, orders)


Unknown SKUs: 0
Unknown customers: 0
Integrity checks completed ..


In [8]:
# Step 8 — Enrichment

def enrich_data(orders: pd.DataFrame) -> pd.DataFrame:
    if orders is None or orders.empty:
        return orders

    o = orders.copy()

    if {'qty', 'unit_price_usd'}.issubset(o.columns):
        o['line_total_usd'] = o['qty'] * o['unit_price_usd']

    if 'order_date' in o.columns:
        try:
            o['order_date_local'] = o['order_date'].dt.tz_convert('Europe/Rome')
        except TypeError:
            o['order_date_local'] = o['order_date'].dt.tz_localize('UTC').dt.tz_convert('Europe/Rome')

        o['order_ym'] = o['order_date_local'].dt.strftime('%Y-%m')

    return o

orders = enrich_data(orders)


In [9]:
# Step 9 — KPIs

orders_kpi = orders.copy()

if {"product_sku"}.issubset(orders_kpi.columns) and {"product_sku","category"}.issubset(products.columns):
    orders_kpi = orders_kpi.merge(
        products[["product_sku", "category"]],
        on="product_sku", how="left"
    )

if {"customer_id"}.issubset(orders_kpi.columns) and {"customer_id","full_name"}.issubset(customers.columns):
    orders_kpi = orders_kpi.merge(
        customers[["customer_id", "full_name"]],
        on="customer_id", how="left"
    )

kpi_revenue_by_month = (
    orders_kpi
    .dropna(subset=["order_ym", "line_total_usd"])
    .groupby("order_ym", as_index=False)["line_total_usd"]
    .sum()
    .rename(columns={"line_total_usd": "revenue_usd"})
    .sort_values("order_ym")
)

kpi_revenue_by_category = (
    orders_kpi
    .dropna(subset=["line_total_usd"])
    .groupby("category", as_index=False)["line_total_usd"]
    .sum()
    .rename(columns={"line_total_usd": "revenue_usd"})
    .sort_values("revenue_usd", ascending=False)
) if "category" in orders_kpi.columns else pd.DataFrame(columns=["category","revenue_usd"])

# KPI 3: top customers (top 5 by revenue)
kpi_top_customers = (
    orders_kpi
    .dropna(subset=["line_total_usd"])
    .groupby(["customer_id","full_name"], as_index=False)
    .agg(orders=("order_date", "size"), revenue_usd=("line_total_usd", "sum"))
    .sort_values("revenue_usd", ascending=False)
    .head(5)
) if {"customer_id","full_name"}.issubset(orders_kpi.columns) else pd.DataFrame(columns=["customer_id","full_name","orders","revenue_usd"])

print("Revenue by Month:")
display(kpi_revenue_by_month)

print("\nRevenue by Category:")
display(kpi_revenue_by_category.head())

print("\nCustomers by Revenue:")
display(kpi_top_customers.head())

Revenue by Month:


,order_ym,revenue_usd
0,2025-02,168.72
1,2025-03,2860.4
2,2025-04,5303.08
3,2025-05,3540.23
4,2025-06,5343.69
5,2025-07,3628.38
6,2025-08,2754.93



Revenue by Category:


,category,revenue_usd
2,Tools,6916.32
1,Gadgets,6543.62
3,Widgets,5890.9
0,Accessories,5507.12



Customers by Revenue:


,customer_id,full_name,orders,revenue_usd
73,C098,Jeffrey Stark Md,4,1082.97
65,C085,Natalie Caldwell,5,1041.01
68,C090,Shelby Cox,3,859.1
40,C054,Jasmine Williams,2,692.2
30,C042,David Davidson Dvm,2,672.98


In [10]:
# Step 10 — Export required CSVs to data/

DATA_DIR.mkdir(parents=True, exist_ok=True)

dim_customers = customers
dim_products  = products
fact_orders   = orders
dim_products_export = dim_products.rename(columns={"product_sku": "sku"})

dim_customers.to_csv(DATA_DIR / "dim_customers_clean.csv", index=False)
dim_products_export.to_csv(DATA_DIR / "dim_products_clean.csv", index=False)
fact_orders.to_csv(DATA_DIR / "fact_orders_clean.csv", index=False)
kpi_revenue_by_month.to_csv(DATA_DIR / "kpi_revenue_by_month.csv", index=False)

print("Exported:")
print("- data/dim_customers_clean.csv")
print("- data/dim_products_clean.csv")
print("- data/fact_orders_clean.csv")
print("- data/kpi_revenue_by_month.csv")


Exported:
- data/dim_customers_clean.csv
- data/dim_products_clean.csv
- data/fact_orders_clean.csv
- data/kpi_revenue_by_month.csv


# ML Task

In [11]:
from pathlib import Path
import pandas as pd
import numpy as np

DATA = Path("data")

fact  = pd.read_csv(DATA / "fact_orders_clean.csv")
prods = pd.read_csv(DATA / "dim_products_clean.csv")
ships = pd.read_csv(DATA / "shipments.csv")

print("fact:", fact.shape, "| prods:", prods.shape, "| ships:", ships.shape)


fact: (200, 10) | prods: (50, 5) | ships: (200, 5)


In [12]:
# Robust mixed-format datetime parser for shipments
from datetime import datetime

def smart_parse(x: str):
    if pd.isna(x): return pd.NaT
    s = str(x).strip()
    if "T" in s or s.endswith("Z") or "+" in s or s.count("-") >= 2:
        return pd.to_datetime(s, errors="coerce", utc=True)
    month_tokens = ["jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]
    if any(m in s.lower() for m in month_tokens):
        return pd.to_datetime(s, errors="coerce", utc=True)
    if "/" in s:
        date_part = s.split()[0]
        parts = date_part.split("/")
        if len(parts[0]) == 4:
            return pd.to_datetime(s, errors="coerce", utc=True)
        else:
            return pd.to_datetime(s, errors="coerce", utc=True, dayfirst=True)
    # Fallback
    return pd.to_datetime(s, errors="coerce", utc=True)

ships_fe = ships.copy()
ships_fe["shipped_at"]   = ships_fe["shipped_at"].apply(smart_parse)
ships_fe["delivered_at"] = ships_fe["delivered_at"].apply(smart_parse)

ships_fe["delivery_days"] = (ships_fe["delivered_at"] - ships_fe["shipped_at"]).dt.total_seconds()/86400.0
ships_fe = ships_fe.loc[ships_fe["delivery_days"].notna() & (ships_fe["delivery_days"] >= 0)].copy()

print("valid shipments:", len(ships_fe))
print(ships_fe["delivery_days"].describe(percentiles=[.05,.5,.95]))

valid shipments: 177
count    177.000000
mean       9.970893
std        2.599782
min        1.994444
5%         4.144296
50%       10.541204
95%       12.887917
max       13.389583
Name: delivery_days, dtype: float64


In [13]:

# join orders with shipments subset >>-- delivered_at not iclude to avoid leakage
xy = fact.merge(
    ships_fe[["order_id", "shipped_at", "carrier", "distance_km", "delivery_days"]],
    on="order_id", how="inner"
)

print("after orders↔shipments:", xy.shape)


after orders↔shipments: (177, 14)


In [14]:
prods = prods.copy()
prods.columns = [c.strip().lower().replace(" ", "_") for c in prods.columns]


keep_prod = [c for c in ["sku","category","weight_kg","unit_price_usd","unit_price","price_usd","price"] if c in prods.columns]

basic_cols = [c for c in ["sku","category","weight_kg"] if c in keep_prod]
xy = xy.merge(prods[basic_cols], left_on="product_sku", right_on="sku", how="left")

price_col = None
for cand in ["unit_price_usd","unit_price","price_usd","price"]:
    if cand in prods.columns:
        price_col = cand
        break

import re
def clean_money(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().replace("$","").replace("USD","").replace(" ", "").replace("'","")
    if "," in s and "." in s:
        s = s.replace(",","")
    elif "," in s and "." not in s:
        s = s.replace(",",".")
    s = re.sub(r"[^0-9\.\-]", "", s)
    try: return float(s)
    except: return np.nan

if price_col is not None:
    price_map = prods[["sku", price_col]].copy()
    price_map["unit_price_usd_clean"] = price_map[price_col].apply(clean_money)
    xy = xy.merge(price_map[["sku","unit_price_usd_clean"]], on="sku", how="left")
    xy = xy.rename(columns={"unit_price_usd_clean":"unit_price_usd"})
else:
    xy["unit_price_usd"] = np.nan

print("after +products:", xy.shape)
print(xy[["product_sku","sku","category","weight_kg","unit_price_usd"]].head())


after +products: (177, 18)
  product_sku      sku     category  weight_kg  unit_price_usd  unit_price_usd
0     SKU-047  SKU-047      Gadgets       0.67           27.36           67.78
1     SKU-023  SKU-023  Accessories       0.44            5.61           54.11
2     SKU-009  SKU-009      Gadgets       0.50           74.33           47.19
3     SKU-017  SKU-017  Accessories       0.27           27.91            5.37
4     SKU-010  SKU-010        Tools       1.33           70.26           74.33


In [15]:
xy["shipped_at"] = pd.to_datetime(xy["shipped_at"], errors="coerce", utc=True)
xy["hour_of_day"] = xy["shipped_at"].dt.hour
xy["weekday"]     = xy["shipped_at"].dt.weekday
print("added time features:", all(c in xy.columns for c in ["hour_of_day","weekday"]))


added time features: True


In [16]:
feature_cols = [
    "ship_country", "qty",
    "category", "weight_kg", "unit_price_usd",
    "carrier", "distance_km", "hour_of_day", "weekday",
]
target_col = "delivery_days"

ids = [c for c in ["order_id", "sku"] if c in xy.columns]
cols = ids + feature_cols + [target_col]
if "product_sku" in xy.columns:
    cols = ids + ["product_sku"] + feature_cols + [target_col]
cols = [c for c in cols if c in xy.columns]

dataset = xy[cols].copy()
dataset = dataset[dataset[target_col].notna() & (dataset[target_col] >= 0)]

out_path = DATA / "ml_dataset.csv"
dataset.to_csv(out_path, index=False)

print(f"[OK] saved -> {out_path}")
print("shape:", dataset.shape)
print("\nFeatures being used for training (excluding product_sku for API compatibility):")
print(f"  {feature_cols}")
print(dataset[target_col].describe(percentiles=[.05,.5,.95]))

[OK] saved -> data/ml_dataset.csv
shape: (177, 14)

Features being used for training (excluding product_sku for API compatibility):
  ['ship_country', 'qty', 'category', 'weight_kg', 'unit_price_usd', 'carrier', 'distance_km', 'hour_of_day', 'weekday']
count    177.000000
mean       9.970893
std        2.599782
min        1.994444
5%         4.144296
50%       10.541204
95%       12.887917
max       13.389583
Name: delivery_days, dtype: float64


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

DATA = Path("data")
df = pd.read_csv(DATA / "ml_dataset.csv").copy()


# df = df[(df["delivery_days"].notna()) & (df["delivery_days"] >= 0) & (df["delivery_days"] <= 60)].copy()
# print("kept rows:", len(df))

df = df[(df["delivery_days"].notna()) & (df["delivery_days"] >= 0)].copy()
print("kept rows (no 60-day filter):", len(df))

features = [c for c in [
    "ship_country", "qty",
    "category", "weight_kg", "unit_price_usd",
    "carrier", "distance_km", "hour_of_day", "weekday",
] if c in df.columns]

print(f"\nUsing features (API-compatible, no product_sku): {features}")

X = df[features]
y = df["delivery_days"]

kept rows (no 60-day filter): 177

Using features (API-compatible, no product_sku): ['ship_country', 'qty', 'category', 'weight_kg', 'unit_price_usd', 'carrier', 'distance_km', 'hour_of_day', 'weekday']


In [18]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

all_null = [c for c in X_train.columns if X_train[c].isna().all()]
if all_null:
    print("Dropping all-null:", all_null)
    X_train = X_train.drop(columns=all_null)
    X_valid = X_valid.drop(columns=[c for c in all_null if c in X_valid.columns])
    features = [c for c in features if c not in all_null]

In [19]:
# 5) preprocessing
num_candidates = ["qty","weight_kg","unit_price_usd","distance_km","hour_of_day","weekday"]
cat_candidates = ["ship_country","category","carrier"]  # NO product_sku here!
num_cols = [c for c in num_candidates if c in X_train.columns]
cat_cols = [c for c in cat_candidates if c in X_train.columns]

print(f"Numerical columns: {num_cols}")
print(f"Categorical columns: {cat_cols}")

try:
    cat_ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)  # sklearn >=1.2
except TypeError:
    cat_ohe = OneHotEncoder(handle_unknown="ignore", sparse=True)         # older sklearn

num_tf = Pipeline([("impute", SimpleImputer(strategy="median")),
                   ("scale",  StandardScaler())])
cat_tf = Pipeline([("impute", SimpleImputer(strategy="most_frequent")),
                   ("onehot", cat_ohe)])

preprocess = ColumnTransformer([
    ("num", num_tf, num_cols),
    ("cat", cat_tf, cat_cols),
], remainder="drop", sparse_threshold=1.0)

Numerical columns: ['qty', 'weight_kg', 'unit_price_usd', 'distance_km', 'hour_of_day', 'weekday']
Categorical columns: ['ship_country', 'category', 'carrier']


In [20]:
# Model A: Linear Regression

pipe_lr = Pipeline([("preprocess", preprocess), ("model", LinearRegression())])

pipe_lr.fit(X_train, y_train)
pred_lr = pipe_lr.predict(X_valid)

mae_lr  = mean_absolute_error(y_valid, pred_lr)
try:
    rmse_lr = mean_squared_error(y_valid, pred_lr, squared=False)
except TypeError:
    rmse_lr = float(np.sqrt(mean_squared_error(y_valid, pred_lr)))
r2_lr   = r2_score(y_valid, pred_lr)

print(f"  LinearRegression -> MAE {mae_lr:.2f} | RMSE {rmse_lr:.2f} | R² {r2_lr:.3f}")


  LinearRegression -> MAE 1.74 | RMSE 2.21 | R² 0.093


In [21]:
def to_numpy_array(X):
    """Convert sparse matrix to a NumPy ndarray (RF needs dense)."""
    return X.toarray() if hasattr(X, "toarray") else np.asarray(X)


In [22]:
from sklearn.ensemble import RandomForestRegressor

pipe_rf  = Pipeline([
    ("preprocess", preprocess),
    ("to_dense", FunctionTransformer(to_numpy_array)),
    ("model", RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)),
])

pipe_rf.fit(X_train, y_train)
pred_rf = pipe_rf.predict(X_valid)

try:
    rmse_rf = mean_squared_error(y_valid, pred_rf, squared=False)
except TypeError:
    rmse_rf = float(np.sqrt(mean_squared_error(y_valid, pred_rf)))
mae_rf  = mean_absolute_error(y_valid, pred_rf)
r2_rf   = r2_score(y_valid, pred_rf)

print(f"RandomForest -> MAE {mae_rf:.2f} | RMSE {rmse_rf:.2f} | R² {r2_rf:.3f}")


RandomForest -> MAE 0.85 | RMSE 1.01 | R² 0.809


In [23]:
# 4) Cross-validation
from sklearn.model_selection import cross_val_score
import numpy as np

cv_mae_lr = -cross_val_score(pipe_lr, X_train, y_train, cv=5,
                             scoring="neg_mean_absolute_error", n_jobs=-1)
cv_mae_rf = -cross_val_score(pipe_rf, X_train, y_train, cv=5,
                             scoring="neg_mean_absolute_error", n_jobs=-1)

print(f"LR  CV MAE: {cv_mae_lr.mean():.3f} ± {cv_mae_lr.std():.3f}")
print(f"RF  CV MAE: {cv_mae_rf.mean():.3f} ± {cv_mae_rf.std():.3f}")


LR  CV MAE: 1.934 ± 0.111
RF  CV MAE: 0.849 ± 0.052


In [24]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

median_y = float(y_train.median())
baseline_pred = np.full(shape=len(y_valid), fill_value=median_y, dtype=float)

mae_base  = mean_absolute_error(y_valid, baseline_pred)
try:
    rmse_base = mean_squared_error(y_valid, baseline_pred, squared=False)
except TypeError:
    rmse_base = float(np.sqrt(mean_squared_error(y_valid, baseline_pred)))
r2_base   = r2_score(y_valid, baseline_pred)

print(f"Baseline (median={median_y:.2f}) -> MAE {mae_base:.2f} | RMSE {rmse_base:.2f} | R² {r2_base:.3f}")


Baseline (median=10.56) -> MAE 1.56 | RMSE 2.35 | R² -0.025


In [25]:
# 6) Explainability: get feature names from the preprocess step
import numpy as np

pre = pipe_lr.named_steps["preprocess"]
try:
    feat_names = pre.get_feature_names_out()
except Exception:
    feat_names = np.array([f"f{i}" for i in range(pre.transform(X_train[:1]).shape[1])])

lr_coef = pipe_lr.named_steps["model"].coef_
lr_top_idx = np.argsort(np.abs(lr_coef))[::-1][:10]
print("\nTop 10 LR coefficients (abs):")
for i in lr_top_idx:
    print(f"{feat_names[i]}: {lr_coef[i]:.4f}")

rf_imp = pipe_rf.named_steps["model"].feature_importances_
rf_top_idx = np.argsort(rf_imp)[::-1][:10]
print("\nTop 10 RF feature importances:")
for i in rf_top_idx:
    print(f"{feat_names[i]}: {rf_imp[i]:.4f}")



Top 10 LR coefficients (abs):
cat__ship_country_SK: -6.8334
cat__ship_country_ES: -6.4440
cat__ship_country_BG: -6.3517
cat__ship_country_IE: -6.2639
cat__ship_country_PL: -6.1524
cat__ship_country_EE: -6.1359
cat__ship_country_DE: -6.0540
cat__ship_country_LV: -5.9075
cat__ship_country_SI: -5.6081
cat__ship_country_CY: -5.5432

Top 10 RF feature importances:
num__distance_km: 0.8558
num__weight_kg: 0.0307
num__weekday: 0.0252
num__hour_of_day: 0.0186
num__unit_price_usd: 0.0142
num__qty: 0.0065
cat__carrier_PosteItaliane: 0.0054
cat__carrier_FedEx: 0.0021
cat__ship_country_AE: 0.0019
cat__category_Widgets: 0.0019


In [26]:
import json

results = {
    "linear_regression": {"mae": float(mae_lr), "rmse": float(rmse_lr), "r2": float(r2_lr)},
    "random_forest":     {"mae": float(mae_rf), "rmse": float(rmse_rf), "r2": float(r2_rf)},
}
best_name = min(results, key=lambda k: results[k]["mae"])
best_pipe = {"linear_regression": pipe_lr, "random_forest": pipe_rf}[best_name]

report = {
    "model": best_name,
    "test_mae": results[best_name]["mae"],
    "test_rmse": results[best_name]["rmse"],
    "r2": results[best_name]["r2"],
}
with open("ml_report.json", "w") as f:
    json.dump(report, f, indent=2)

df_all = pd.read_csv("data/ml_dataset.csv")

# df_f = df_all[(df_all["delivery_days"].notna()) & (df_all["delivery_days"] >= 0) & (df_all["delivery_days"] <= 60)].copy()

df_f = df_all[(df_all["delivery_days"].notna()) & (df_all["delivery_days"] >= 0)].copy()

ids_valid = df_f.loc[X_valid.index, "order_id"] if "order_id" in df_f.columns else pd.Series(range(len(X_valid)))

pred_best = best_pipe.predict(X_valid)
pred_df = pd.DataFrame({
    "order_id": ids_valid.values,
    "y_true": y_valid.values,
    "y_pred": pred_best.astype(float),
})
pred_df.to_csv("predictions.csv", index=False)


print("[OK] Wrote ml_report.json and predictions.csv")
print("=== Final Report ===")
print(json.dumps(report, indent=2))

[OK] Wrote ml_report.json and predictions.csv
=== Final Report ===
{
  "model": "random_forest",
  "test_mae": 0.8491773716135125,
  "test_rmse": 1.0134884605031975,
  "r2": 0.8089853933497675
}


# Model Export Required For Backend Task


In [27]:
import joblib

MODEL_VERSION = "v1"

trained_preprocessor = best_pipe.named_steps["preprocess"]
trained_model = best_pipe.named_steps["model"]

backend_pipeline = Pipeline([
    ('preprocessor', trained_preprocessor),
    ('regressor', trained_model)
])

MODELS = Path("models")
MODELS.mkdir(exist_ok=True) 

backend_model_path = MODELS / f"{MODEL_VERSION}.joblib"
joblib.dump(backend_pipeline, backend_model_path)

print(f"model saved: {backend_model_path}")
print(f"Model version: {MODEL_VERSION}")

model saved: models/v1.joblib
Model version: v1


In [28]:
# to test the model prediction 
test_df = pd.DataFrame({
    'qty': [2],
    'weight_kg': [0.8],
    'unit_price_usd': [12.5],
    'distance_km': [350.0],
    'hour_of_day': [14],
    'weekday': [2],
    'ship_country': ['IT'],
    'category': ['Widgets'],
    'carrier': ['DHL']
})

backend_model = joblib.load(backend_model_path)
prediction = backend_model.predict(test_df)

print("API Contract Input:")
print(test_df.iloc[0].to_dict())
print(f"\n Predicted delivery days: {prediction[0]:.2f}")

API Contract Input:
{'qty': 2, 'weight_kg': 0.8, 'unit_price_usd': 12.5, 'distance_km': 350.0, 'hour_of_day': 14, 'weekday': 2, 'ship_country': 'IT', 'category': 'Widgets', 'carrier': 'DHL'}

 Predicted delivery days: 4.50


In [30]:
print("ho")

ho
